# Word Embedding

In [1]:
"""
    @Project: nlp-learning-tutorials
    @File   : Embedding.ipynb
    @Author : Hivan Du
    @E-mail : doo@hivan.me
    @Wechat : 坍缩的奇点
    @Date   : 2023-12-24 15:35
"""
import os
print(os.getcwd())
import gensim
print(gensim.__version__)

/Volumes/Library/git/AI_Cheats/Core BI
4.3.2


In [2]:
import jieba
import os
# utils 为自定义的一套程序
from utils import files_processing
from gensim.models import word2vec
import multiprocessing

## Word segment

In [3]:
# 源文件所在目录
source_folder = 'dataset/journey_to_the_west/source'
segment_folder = 'dataset/journey_to_the_west/segment'

In [4]:
# 字词分割，对整个文件内容进行字词分割
def segment_lines(file_list,segment_out_dir,stopwords=[]):
    for i,file in enumerate(file_list):
        segment_out_name=os.path.join(segment_out_dir,'segment_{}.txt'.format(i))
        with open(file, 'rb') as f:
            document = f.read()
            document_cut = jieba.cut(document)
            sentence_segment=[]
            for word in document_cut:
                if word not in stopwords:
                    sentence_segment.append(word)
            result = ' '.join(sentence_segment)
            result = result.encode('utf-8')
            with open(segment_out_name, 'wb') as f2:
                f2.write(result)

In [5]:
# 对 source 中的 txt 文件进行分词，输出到 segment 目录中
file_list=files_processing.get_files_list(source_folder, postfix='*.txt')
segment_lines(file_list, segment_folder)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/h4/7cr1cmpn7v5b3x20_9wz8m740000gn/T/jieba.cache


Loading model cost 0.344 seconds.
Prefix dict has been built successfully.


## Word similarity

In [6]:
# 如果目录中有多个文件，可以使用 PathLineSentences
segment_folder = 'dataset/journey_to_the_west/segment'
sentences = word2vec.PathLineSentences(segment_folder)

In [7]:
# 设置模型参数，进行训练
model = word2vec.Word2Vec(sentences, vector_size=100, window=3, min_count=1)
print(model.wv.similarity('孙悟空', '猪八戒'))
print(model.wv.similarity('孙悟空', '孙行者'))
print(model.wv.most_similar(positive=['孙悟空', '唐僧'], negative=['孙行者']))

0.933632
0.9586093
[('老高', 0.9716116189956665), ('者', 0.9710014462471008), ('上前', 0.9701845645904541), ('菩萨', 0.9676584005355835), ('二哥', 0.9670479893684387), ('跪下', 0.9643738865852356), ('我来', 0.9643673896789551), ('大王', 0.9637192487716675), ('悟空', 0.9630470871925354), ('长老', 0.9616209864616394)]


In [8]:
# 设置模型参数，进行训练
model2 = word2vec.Word2Vec(sentences, vector_size=128, window=5, min_count=5, workers=multiprocessing.cpu_count())

In [9]:
# 保存模型
model2.save('dataset/models/word2Vec.model')

In [10]:
print(model2.wv.similarity('孙悟空', '猪八戒'))
print(model2.wv.similarity('孙悟空', '孙行者'))
print(model2.wv.most_similar(positive=['孙悟空', '唐僧'], negative=['孙行者']))

0.8977146
0.9110702
[('菩萨', 0.9150608777999878), ('老高', 0.9012092351913452), ('弟子', 0.8970403671264648), ('请', 0.8915565013885498), ('扯住', 0.8880948424339294), ('众', 0.8798324465751648), ('列位', 0.8727977871894836), ('老爷', 0.8690122961997986), ('武洗', 0.8664023876190186), ('跪下', 0.8619289398193359)]
